In [1]:
import pandas as pd
import numpy as np
import os
import re
from utils import *
import sounddevice as sd
import soundfile as sf

from keras.utils import to_categorical
from keras.optimizers import Adam

import tensorflow as tf

import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel

import matplotlib.pyplot as plt


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

MAX_SEQUENCE_LENGTH = 66
BATCH_SIZE = 32
CATEGORIES = "sadness,joy,anger,fear,surprise,neutral".split(',')


def tokenize(sentences, max_length=MAX_SEQUENCE_LENGTH, padding='max_length'):

    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

def decode(tokens):
    return tokenizer.decode(tokens)

def to_tensor(data, label = []):
    return tf.data.Dataset.from_tensor_slices((
    dict(tokenize(data)),  # Convert BatchEncoding instance to dictionary
    label
    )).batch(BATCH_SIZE).prefetch(1)

def predict(str,model):
    str = to_tensor(str)
    model.predict(str)

def predice(array,model):

    array = tf.data.Dataset.from_tensor_slices((
    dict(tokenize(array)),
    )).batch(BATCH_SIZE).prefetch(1)
    df = pd.DataFrame(columns=CATEGORIES,data=model.predict(array))
    #reorder the columns in the following order: surprise, anger, fear, sadness, joy, neutral
    df = df[['surprise','anger','fear','sadness','joy','neutral']]
    return df, df.values



new_model =  tf.keras.models.load_model('C:\\Users\\mario\\PROYECTO_IA\\IA-project\\Notebooks\\MARIO\\SentimentAnalysis.hdf5', custom_objects={'TFDistilBertModel': TFDistilBertModel})


c:\Users\mario\anaconda3\envs\CAPSTONE_PROJECT\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df, inputs = predice("She was feeling rather sick",new_model)

inputs = inputs[0]

generate_sound(get_notes(inputs))
df

1/1 [==============================] - 4s 4s/step


ValueError: Shape of passed values is (1, 5), indices imply (1, 6)

: 

In [ ]:
filename = 'PRUEBAS2.wav'
# Extract data and sampling rate from file
data, fs = sf.read(filename, dtype='float32')  
sd.play(data, fs)
status = sd.wait()  # Wait until file is done playing

array([[0.01574196, 0.44432044, 0.02142012, 0.31377485, 0.0068816 ,
        0.19786099]], dtype=float32)